<a href="https://colab.research.google.com/github/olinml2024/notebooks/blob/main/ML24_Assignment10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Dataset: https://ai.stanford.edu/~amaas/data/sentiment/

In [ ]:
import gdown

gdown.download(id='1AhSDUrPbRPS0JorkDR8W1ZoyeFs8vF55', output='aclImdb_v1.tar.gz')
!tar -xvzf aclImdb_v1.tar.gz > /dev/null 2>&1

Downloading...
From (original): https://drive.google.com/uc?id=1AhSDUrPbRPS0JorkDR8W1ZoyeFs8vF55
From (redirected): https://drive.google.com/uc?id=1AhSDUrPbRPS0JorkDR8W1ZoyeFs8vF55&confirm=t&uuid=4153a131-84c0-4e8d-b958-8f7b97ec4459
To: /content/aclImdb_v1.tar.gz
100%|██████████| 84.1M/84.1M [00:00<00:00, 179MB/s]


In [ ]:
import glob

positive_reviews = []

for file in glob.glob('aclImdb/train/pos/*'):
    with open(file) as f:
        positive_reviews.append(f.read())

negative_reviews = []

for file in glob.glob('aclImdb/train/neg/*'):
    with open(file) as f:
        negative_reviews.append(f.read())

print(f"number of positive reviews {len(positive_reviews)}")
print(f"number of negative reviews {len(negative_reviews)}")

number of positive reviews 12500
number of negative reviews 12500


Let's take a look at some of the positive reviews.

In [ ]:
import random
import textwrap

for review in random.choices(positive_reviews, k=5):
    print("POSITIVE REVIEW:")
    print('\n'.join(textwrap.wrap(review, 50, break_long_words=False)))

REVIEW:
Diane and I saw this fabulous film today in
Fremantle and we both agreed that of the pastiche
movies it was head and shoulders above the rest. I
say that because we were entranced by the brief,
five to ten minute segments that composed the film
and the fact that this film had a theme around
which each piece was composed and of course that
theme was love in its many forms.<br /><br
/>Ostensibly the film took place in the various
Parisian arrondisments thus giving a particular
flavour to each segment. Having only been in Paris
several times, I was not knowledgeable enough to
readily recognize the locations but I am sure
Europeans and particularly French people could
easily recognize the city's locations. In any
event, the viewer is immediately pulled into each
story because of their production excellence so
these city locations fade into in-consequence.<br
/><br />The film moves quickly and the viewer is
left absorbing one scenario while the new one is
on the screen. The stories 

And some of the negative reviews.

In [ ]:
import random
import textwrap

for review in random.choices(negative_reviews, k=5):
    print("NEGATIVE REVIEW:")
    print('\n'.join(textwrap.wrap(review, 50, break_long_words=False)))

NEGATIVE REVIEW:
Welcome to movie 17 on the chilling classics 50
pack. Where we'll see, That's right. Another movie
that makes absolutely no sense. Seriously, this
movie had me so confused at the end, i thought i
was rewatching "At Dawn they Sleep." The plot
seems simple enough....well that is until 3
seconds into the movie where a girl supposedly
killed a cat and then...um.. explodes? i have no
idea what happened. and that was BEFORE THE TITLE
SCREEN. That's really sad when i can't even tell
what happened in the first 3 minutes.<br /><br
/>Anyway it stars a photographer with a big
mustache who finds this girl after dumping his
other girlfriend on the way to take pictures of
something somewhere. so we get there but not
before somebody steals their jeep to drive it 200
feet out of the way towards a town. suspicious?
nah. so they decide to stay at this deserted
village with one old lady. and then blah blah
stuff happens and blah blah talking. The guy with
the mustache goes out in the fog

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(positive_reviews + negative_reviews)
y = [1] * len(positive_reviews) + [0] * len(negative_reviews)

In [ ]:
row_ind = 10000
bow_as_dict = dict(zip(vectorizer.get_feature_names_out()[X.getrow(row_ind).indices].tolist(),
                       X.getrow(row_ind).data.tolist())
                  )
print(bow_as_dict)


{'outstanding': 1, 'the': 12, 'female': 2, 'role': 1, 'in': 3, 'out': 1, 'and': 8, 'to': 7, 'it': 3, 'my': 1, 'is': 7, 'more': 1, 'then': 1, 'which': 1, 'also': 1, 'with': 2, 'how': 1, 'have': 3, 'so': 3, 'good': 2, 'you': 3, 'if': 1, 'this': 5, 'not': 1, 'for': 4, 'of': 3, 'but': 1, 'at': 1, 'these': 2, 'one': 1, 'well': 2, 'before': 1, 'film': 1, 'be': 1, 'that': 5, 'story': 2, 'all': 3, 'think': 1, 'by': 1, 'seen': 1, 'movie': 4, 'from': 1, 'acting': 1, 'low': 2, 'budget': 2, 'about': 1, 'times': 1, 'man': 1, 'see': 2, 'into': 1, 'because': 1, 'legal': 1, 'are': 2, 'too': 1, 'far': 1, 'do': 1, 'as': 1, 'been': 1, 'actor': 1, 'script': 1, 'some': 2, 'we': 1, 'law': 1, 'explains': 1, 'great': 2, 'an': 3, 'up': 1, 'plot': 1, 'takes': 1, 'did': 1, 'interesting': 1, 'main': 2, 'could': 1, 'lot': 1, 'action': 2, 'flick': 2, 'only': 1, 'beyond': 1, 'want': 1, 'me': 1, 'go': 1, 'music': 1, 'them': 1, 'get': 2, 'original': 1, 'something': 1, 'there': 2, 'sexy': 2, 'should': 1, 'perfect': 1, 

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X, y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
import numpy as np

print("Predictive of a bad review")
for idx in np.argsort(model.coef_[0])[:10]:
    print(vectorizer.get_feature_names_out()[idx])

print("Predictive of a good review")
for idx in np.argsort(model.coef_[0])[-10:]:
    print(vectorizer.get_feature_names_out()[idx])

Predictive of a bad review
waste
disappointment
poorly
lacks
mess
worst
disappointing
fails
awful
avoid
Predictive of a good review
gem
subtle
enjoyable
incredible
superb
surprisingly
rare
funniest
refreshing
wonderfully
